In [9]:
pip install langchain faiss-cpu pymupdf requests langchain-community


Note: you may need to restart the kernel to use updated packages.


Load and Preprocess PDFs:

In [2]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#prompt for pdf file path
# pdf_file_path = input("Enter the path to the PDF file: ")
pdf_file_path="/home/axat/personal/rag-playground/Nvidia-NIM/us_census/acsbr-015.pdf"
# Load PDF
loader = PyMuPDFLoader(pdf_file_path)
documents = loader.load()

# print(documents)

# Split text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter()
texts = text_splitter.split_documents(documents)

#print all the text
print(texts)

[Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern\xad\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Columbia can affect health insurance \ncoverage by making Marketplace or Med

Embed Text using LLAMA3

In [14]:
# from langchain_community.embeddings import OllamaEmbeddings

# # Initialize OllamaEmbeddings
# ollama_emb = OllamaEmbeddings(
#     model="llama3")
# # Embed the documents
# embeddings = ollama_emb.embed_documents(texts)
# print(embeddings)

[[2.9329967498779297, -4.9099297523498535, -3.1031813621520996, -1.7397440671920776, -1.708496332168579, -2.8366246223449707, -2.514166831970215, 2.403798818588257, 1.3315690755844116, -1.3232331275939941, -0.9989519715309143, -0.6255924105644226, -6.3965582847595215, 4.04596471786499, 4.403615951538086, 2.3585166931152344, -1.594760775566101, 2.3606624603271484, -1.48470139503479, 0.921811580657959, -1.1089348793029785, 1.0978959798812866, 0.029308171942830086, 0.9792520999908447, -2.6129088401794434, 4.031653881072998, -2.6267480850219727, 1.6211155652999878, 5.1460280418396, -2.262237548828125, -1.9058831930160522, 4.493139743804932, -2.0250747203826904, 4.2186598777771, -2.6959781646728516, -2.3404016494750977, 1.960698127746582, 0.2790032923221588, 3.202939033508301, 4.214776515960693, 2.5073118209838867, -2.109431743621826, 1.498518466949463, 1.1774423122406006, -1.7434577941894531, 0.8075150847434998, 0.9392715692520142, 0.2920771837234497, -1.5297279357910156, 0.062055431306362

Index Embeddings with FAISS:

we need to use embedding from ollamaEnbeddings 


In [18]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Convert texts to a list of strings
text_contents = [str(text) for text in texts]

embeddings = OllamaEmbeddings(model="llama3",temperature=0)

# Create FAISS vector store
faiss_store = FAISS.from_texts(text_contents, embeddings)

# Save the vector store
faiss_store.save_local("faiss_store")

Create LangChain Components : retriveal

In [2]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="llama3", temperature=0)

# Load the FAISS vector store
faiss_store = FAISS.load_local(
    "faiss_store", embeddings, allow_dangerous_deserialization=True)

# Create a retriever
retriever = faiss_store.as_retriever()


# Use the retriever to find relevant documents
query = "WHAT IS HEALTH INSURANCE COVERAGE?"
docs = retriever.invoke(query)

# docs_and_scores = faiss_store.similarity_search_with_score(query)
# print(docs_and_scores)


embedding_vector = embeddings.embed_query(query)
docs_and_scores = faiss_store.similarity_search_by_vector(embedding_vector)

print(docs_and_scores)
# Print retrieved documents
# for doc in docs:
#     print(doc)

[Document(page_content='page_content=\'Health Insurance Coverage Status and Type \\nby Geography: 2021 and 2022\\nAmerican Community Survey Briefs\\nACSBR-015\\nIssued September 2023\\nDouglas Conway and Breauna Branch\\nINTRODUCTION\\nDemographic shifts as well as economic and govern\\xad\\nment policy changes can affect people’s access to \\nhealth coverage. For example, between 2021 and 2022, \\nthe labor market continued to improve, which may \\nhave affected private coverage in the United States \\nduring that time.1 Public policy changes included \\nthe renewal of the Public Health Emergency, which \\nallowed Medicaid enrollees to remain covered under \\nthe Continuous Enrollment Provision.2 The American \\nRescue Plan (ARP) enhanced Marketplace premium \\nsubsidies for those with incomes above 400 percent \\nof the poverty level as well as for unemployed people.3\\nIn addition to national policies, individual states and \\nthe District of Columbia can affect health insurance \\n

grader system for grading 

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="llama3", temperature=0)

# Load the FAISS vector store
faiss_store = FAISS.load_local(
    "faiss_store", embeddings, allow_dangerous_deserialization=True)


# Initialize the ChatOllama model
llm = ChatOllama(model="llama3", format="json", temperature=0)

# Define the prompt template
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

# Define your retriever here
retriever = faiss_store.as_retriever()

# Create the retrieval grader
retrieval_grader = prompt | llm | JsonOutputParser()

# Define your question
question = "WHAT IS HEALTH INSURANCE COVERAGE"

# Retrieve the documents
docs = retriever.invoke(question)

# Get the text of the first document
doc_txt = docs[1].page_content

# Invoke the retrieval grader
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}
